In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import seaborn as sns
from tqdm.notebook import tqdm

from pprint import pprint

from utils import *

# Find Files

In [ ]:
md_files = [
    '/home/wmnlab/D/database/2023-08-29/2023-08-29.md', 
    '/home/wmnlab/D/database/2023-09-12-1/2023-09-12-1.md',
    '/home/wmnlab/D/database/2023-09-22/2023-09-22.md',
    '/home/wmnlab/D/database/2023-10-24/2023-10-24.md',
    # '/home/wmnlab/D/database/2023-11-21/2023-11-21.md',
    # '/home/wmnlab/D/database/2023-12-26/2023-12-26.md',
    '/home/wmnlab/D/database/2024-03-10/2024-03-10.md',
    '/home/wmnlab/D/database/2024-03-16/2024-03-16.md',
    '/home/wmnlab/E/database/2024-04-18/2024-04-18.md',
    '/home/wmnlab/E/database/2024-05-08/2024-05-08.md'
    ]

EXPs = get_EXPs(md_files)
pprint(EXPs)

# Overlapped RLF analysis

In [ ]:
CG1_DL, CG2_DL, CG3_DL, v3_DL, v4_DL = [], [], [], [], []
CG1_HO, CG2_HO, CG3_HO, v3_HO, v4_HO = [], [], [], [], []

for exp in tqdm(EXPs):
    if exp.type in ['Modem_Action_Test', 'UDP_Static_Bandlock']:
        continue
    dev_dir_list = find_device_under_exp(exp.path)
    [dev1, dev2] = list(exp.settings.keys())
    dev_dir_list1 = [d for d in dev_dir_list if dev1 in d]
    dev_dir_list2 = [d for d in dev_dir_list if dev2 in d]
    dev_dir_path1 = dev_dir_list1[0]
    dev_dir_path2 = dev_dir_list2[0]

    trace_dir_list1 = find_trace_under_device(dev_dir_path1)
    trace_dir_list2 = find_trace_under_device(dev_dir_path2)
    for t1, t2 in tqdm(zip(trace_dir_list1, trace_dir_list2), leave=False):
        rrc1, rrc2 = return_rrc(t1), return_rrc(t2)
        DL1, DL2 = return_DL(t1), return_DL(t2)
        UL1, UL2 = return_UL(t1), return_UL(t2)

        HOs1, HOs2 = parse_mi_ho(rrc1), parse_mi_ho(rrc2)
        loss, excl = loss_excl_cause_dual(DL1, DL2, rrc1, rrc2)
        CG1_HO.append((HOs1, HOs2))
        CG1_DL.append(loss)

        if exp.type == 'Control_Group':
            CG1_HO.append((HOs1, HOs2))
            CG1_DL.append(loss)
        if exp.type == 'Control_Group2':
            CG2_HO.append((HOs1, HOs2))
            CG2_DL.append(loss)
        if exp.type == 'Control_Group3':
            CG3_HO.append((HOs1, HOs2))
            CG3_DL.append(loss)
        if exp.type == 'Modem_Action_Test_v3':
            v3_HO.append((HOs1, HOs2))
            v3_DL.append(loss)
        if exp.type == 'Modem_Action_Test_v4':
            v4_HO.append((HOs1, HOs2))
            v4_DL.append(loss)

In [ ]:
def count_overlapped_RLF(CG_HO, CG_DL):
    overlapped_RLF = 0
    loss_packet_nums = 0
    diffs = []
    for (hos1, hos2), loss in zip(CG_HO, CG_DL):
        rlfs1 = hos1['RLF_II'] + hos1['RLF_III']
        rlfs2 = hos2['RLF_II'] + hos2['RLF_III']
        for rlf1 in rlfs1:
            for rlf2 in rlfs2:
                diff = (rlf1.start - rlf2.start).total_seconds()
                if abs(diff) < 3.0:
                    print(rlf1)
                    print(rlf2)
                    overlapped_RLF += 1
                    diffs.append(abs(diff))
        for l in loss:
            diff1 = 10000 # infinity
            for rlf1 in rlfs1:
                diff = (rlf1.start - l.timestamp1).total_seconds()
                diff1 = abs(diff) if abs(diff) < diff1 else diff1
            diff2 = 10000 # infinity
            for rlf2 in rlfs2:
                diff = (rlf2.start - l.timestamp1).total_seconds()
                diff2 = abs(diff) if abs(diff) < diff2 else diff2
            diff2 = (rlf2.start - l.timestamp2).total_seconds()
            if diff1 < 4.0 or diff2 < 4.0:
                loss_packet_nums += 1
    
    return overlapped_RLF, loss_packet_nums, diffs

overlapped_RLF1, loss_packet_nums1, diffs1 = count_overlapped_RLF(CG1_HO, CG1_DL)
overlapped_RLF2, loss_packet_nums2, diffs2 = count_overlapped_RLF(CG2_HO, CG2_DL)
overlapped_RLF3, loss_packet_nums3, diffs3 = count_overlapped_RLF(CG3_HO, CG3_DL)
overlapped_RLF4, loss_packet_nums4, diffs4 = count_overlapped_RLF(v3_HO, v3_DL)
overlapped_RLF5, loss_packet_nums5, diffs5 = count_overlapped_RLF(v4_HO, v4_DL)

In [ ]:
print(overlapped_RLF1, loss_packet_nums1, diffs1)
print(overlapped_RLF2, loss_packet_nums2, diffs2)
print(overlapped_RLF3, loss_packet_nums3, diffs3)
print(overlapped_RLF4, loss_packet_nums4, diffs4)
print(overlapped_RLF5, loss_packet_nums5, diffs5)

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

x = [1,2,3,4]
x2 = [i+0.4 for i in x]   # 因為長條圖寬度 0.4，所以位移距離為除以 2 為 0.2
ax1.bar(x,height=[34/10,2/10,7/12,1/12],width=0.4, align='center', label='Average overlapped RLF happen times')
ax2.bar(x2,height=[10904/34,2059/2,786/7,197/1], align='center', color='orange',width=0.4, label='Packet loss per overlapped RLF')

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, bbox_to_anchor=(1.1, 1.15))
ax1.set_ylabel('Happen Times')
ax2.set_ylabel('Lost Packet Nums')

plt.xticks([i+0.2 for i in x], ['5G+5G', '5G+LTE', 'B1B3+B7B8', 'DBL'], size='small')

plt.show()

# System Packet Loss Plot

In [ ]:
# Get names of files
date = '2024-01-08'
exp = 'Modem_Action_Test_v4'
trace = '#03'

base_dir = '/home/wmnlab/E/database'
date_dir = os.path.join(base_dir, date)
d = os.path.join(date_dir, exp)
dev1, dev2 = 'qc00', 'qc01'
trace_dir1 = os.path.join(d, dev1, trace)
trace_dir2 = os.path.join(d, dev2, trace)
rrc_file1 = return_rrc(trace_dir1)
rrc_file2 = return_rrc(trace_dir2)
rrc_file1, rrc_file2

In [ ]:
HOs = parse_mi_ho(rrc_file1)
MRs = MeasureReport(rrc_file1)
MRs = correct_MR_with_HO(MRs, HOs)
mappings = map_MR_HO(MRs, HOs)
ordered_HOs1 = print_trans(HOs, mappings=mappings)
print('=================================')
HOs = parse_mi_ho(rrc_file2)
MRs = MeasureReport(rrc_file2)
MRs = correct_MR_with_HO(MRs, HOs)
mappings = map_MR_HO(MRs, HOs)
ordered_HOs2 = print_trans(HOs, mappings=mappings)

In [ ]:
dl_loss_excl_file1 = os.path.join(trace_dir1, 'data', 'udp_dnlk_loss_latency.csv')
dl_loss_excl_file2 = os.path.join(trace_dir2, 'data', 'udp_dnlk_loss_latency.csv')
dl_loss1, dl_excl1 = loss_excl_cause(dl_loss_excl_file1, rrc_file1)
dl_loss2, dl_excl2 = loss_excl_cause(dl_loss_excl_file2, rrc_file2)
dl_loss, dl_excl = loss_excl_cause_dual(dl_loss_excl_file1, dl_loss_excl_file2, rrc_file1, rrc_file2)

In [ ]:
dl_loss

In [ ]:
# Read time sync file
time_sync_file = [os.path.join(date_dir, x) for x in os.listdir(date_dir) if x.endswith('.json') and 'time_sync' in x][0]
print(time_sync_file)
with open(time_sync_file, 'r') as f:
    time_off_dict = json.load(f)

pattern = r'(\d{4}-\d+-\d+_\d{2}-\d{2}-\d{2})'
match = re.search(pattern, rrc_file1)
if match:
    datetime_str = match.group(1)
    filetime = pd.to_datetime(datetime_str, format='%Y-%m-%d_%H-%M-%S')
else:
    print("File Name Error!"); raise

measure_times = [pd.to_datetime(k, format='%Y-%m-%d %H:%M:%S.%f') for k in time_off_dict.keys()]
time_diffs = [abs(t-filetime) for t in measure_times]
ind, value = min(enumerate(time_diffs), key=lambda x: x[1])
print('rrc file time:', filetime)
delta = dt.timedelta(seconds=list(time_off_dict.values())[ind])
print('Sync Time:', list(time_off_dict.keys())[ind], '; delta',list(time_off_dict.values())[ind])

In [ ]:
# dl_loss1
# R1_dl_pkgs = accumulate_packet(dl_loss_excl_file1)
# R2_dl_pkgs = accumulate_packet(dl_loss_excl_file2)

start_time = dt.datetime(2024, 4, 18, 18, 42, 45, 000000)
end_time = dt.datetime(2024, 4, 18, 18, 42, 55, 000000)
time_range = (start_time, end_time)

In [ ]:
# Read TimeSync csv
dev_dir = os.path.join(date_dir, 'others/TimeSync', dev1)
files = os.listdir(dev_dir)
value = max(files, key = lambda x: find_longest_common_substring_length(x, rrc_file1))
ind = files.index(value)
TimeSync_file1 = os.path.join(dev_dir, files[ind])

dev_dir = os.path.join(date_dir, 'others/TimeSync', dev2)
files = os.listdir(dev_dir)
value = max(files, key = lambda x: find_longest_common_substring_length(x, rrc_file2))
ind = files.index(value)
TimeSync_file2 = os.path.join(dev_dir, files[ind])

print(rrc_file1, TimeSync_file1, rrc_file2, TimeSync_file2, sep='\n')

TS_df1 = pd.read_csv(TimeSync_file1, names=['cell time', 'device time'])
TS_df1['cell time'] = pd.to_datetime(TS_df1['cell time']) + dt.timedelta(hours=8)
TS_df1['device time'] = pd.to_datetime(TS_df1['device time'])

TS_df2 = pd.read_csv(TimeSync_file2, names=['cell time', 'device time'])
TS_df2['cell time'] = pd.to_datetime(TS_df2['cell time']) + dt.timedelta(hours=8)
TS_df2['device time'] = pd.to_datetime(TS_df2['device time'])

In [ ]:
# Align ordered_HOs HO time to server time
ordered_HOs1_ = ordered_HOs1 # store previous value
ordered_HOs2_ = ordered_HOs2 # store previous value
ordered_HOs1 = ho_time_to_server_time(ordered_HOs1, TS_df1, delta)
ordered_HOs2 = ho_time_to_server_time(ordered_HOs2, TS_df2, delta)

In [ ]:
T1, Type1, Trans1, Ev1 = get_info(ordered_HOs1, time_range)
T2, Type2, Trans2, Ev2 = get_info(ordered_HOs2, time_range)

fig, ax = plt.subplots()
HO_SCA1 = plt.scatter(T1, [0.5] * len(T1), marker='*', color='b', s=20, zorder=3, label='R1 HO')
HO_SCA2 = plt.scatter(T2, [-0.5] * len(T2), marker='*', color='r', s=20, zorder=3, label='R2 HO')

plt.axhline(y=0, color='gray', linestyle='--', linewidth=1)

# Loss
L1 = [pkg.timestamp for pkg in dl_loss1]
L2 = [pkg.timestamp for pkg in dl_loss2]
DL_LOSS1 = ax.vlines(x=L1, ymin=0, ymax=0.3, alpha=1, zorder=2, colors='lightblue', label='R1 DL Loss')
DL_LOSS2 = ax.vlines(x=L2, ymin=-0.3, ymax=0, alpha=1, zorder=2, colors='lightpink', label='R2 DL Loss')

# Latency
# R1_Lat = ax.plot([pkg.Timestamp for pkg in R1_dl_pkgs], [pkg.latency for pkg in R1_dl_pkgs], 
#             zorder=1, label='R1 DL Latency', linestyle='-')[0]
# R2_Lat = ax.plot([pkg.Timestamp for pkg in R2_dl_pkgs], [-pkg.latency for pkg in R2_dl_pkgs], 
#             zorder=1, label='R2 DL Latency'
#             , linestyle='-')[0]

ax.set_xlim(time_range)
# x_ticks = ax.get_xticks()
# x_lim = ax.get_xlim()
# ax.set_xticks(x_ticks)
# ax.set_xticklabels(list(range(0,len(x_ticks)*60,60)))
# plt.xlabel('5 minute timeline (sec)')

plt.ylim([-0.8, 0.8])
ax.set_yticks([-0.5, 0, 0.5])
ax.set_yticklabels(['0.5','0','0.5'])
plt.ylabel('latency (sec)')

ax.legend(bbox_to_anchor=(1.35, 1.))
plt.show()

# Performace

In [ ]:
keys = ['DL Loss','DL Excl','UL Loss','UL Excl']
CG1 = {k:[] for k in keys}
CG2 = {k:[] for k in keys}
CG3 = {k:[] for k in keys}
CG5 = {k:[] for k in keys}
v3 = {k:[] for k in keys}
v4 = {k:[] for k in keys}
v6 = {k:[] for k in keys}
for exp in tqdm(EXPs):
    if exp.type in ['Modem_Action_Test', 'UDP_Static_Bandlock']:
        continue
    dev_dir_list = find_device_under_exp(exp.path)
    [dev1, dev2] = list(exp.settings.keys())
    dev_dir_list1 = [d for d in dev_dir_list if dev1 in d]
    dev_dir_list2 = [d for d in dev_dir_list if dev2 in d]
    dev_dir_path1 = dev_dir_list1[0]
    dev_dir_path2 = dev_dir_list2[0]

    trace_dir_list1 = find_trace_under_device(dev_dir_path1)
    trace_dir_list2 = find_trace_under_device(dev_dir_path2)
    for t1, t2 in tqdm(zip(trace_dir_list1, trace_dir_list2), leave=False):
        rrc1, rrc2 = return_rrc(t1), return_rrc(t2)
        DL1, DL2 = return_DL(t1), return_DL(t2)
        UL1, UL2 = return_UL(t1), return_UL(t2)

        # HOs1, HOs2 = parse_mi_ho(rrc1), parse_mi_ho(rrc2)
        dl_loss, dl_excl = count_loss_excl_rate_dual(DL1, DL2)
        ul_loss, ul_excl = count_loss_excl_rate_dual(UL1, UL2)
        metrics = [dl_loss*100, dl_excl*100, ul_loss*100, ul_excl*100]

        if exp.type == 'Control_Group':
            for k, v in zip(keys, metrics):
                CG1[k].append(v)
        if exp.type == 'Control_Group2':
            for k, v in zip(keys, metrics):
                CG2[k].append(v)
        if exp.type == 'Control_Group3':
            for k, v in zip(keys, metrics):
                CG3[k].append(v)    
        if exp.type == 'Control_Group5':
            for k, v in zip(keys, metrics):
                CG5[k].append(v)
        if exp.type == 'Modem_Action_Test_v3':
            for k, v in zip(keys, metrics):
                v3[k].append(v)
        if exp.type == 'Modem_Action_Test_v4':
            for k, v in zip(keys, metrics):
                v4[k].append(v)
        if exp.type == 'Modem_Action_Test_v6':
            for k, v in zip(keys, metrics):
                v6[k].append(v)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2)

data_A = [CG1['DL Loss']*100, CG2['DL Loss']*100, CG3['DL Loss']*100, CG5['DL Loss']*100, v6['DL Loss']*100]
data_B = [CG1['DL Excl']*100, CG2['DL Excl']*100, CG3['DL Excl']*100, CG5['DL Excl']*100, v6['DL Excl']*100]
data_C = [CG1['UL Loss']*100, CG2['UL Loss']*100, CG3['UL Loss']*100, CG5['UL Loss']*100, v6['UL Loss']*100]
data_D = [CG1['UL Excl']*100, CG2['UL Excl']*100, CG3['UL Excl']*100, CG5['UL Excl']*100, v6['UL Excl']*100]
datas = [data_A, data_B, data_C, data_D]
labels = keys

for row, col, data, k in zip([0,0,1,1], [0,1,0,1], datas, keys):
    axes[row, col].boxplot(data)
    means = [np.mean(d) for d in data]
    for i, mean in enumerate(means, 1):
        axes[row, col].scatter(i, mean, color='red', zorder=3, s=20) 
        axes[row, col].text(i, mean, f'{mean:.3f}', color='r', ha='center', va='bottom')
    axes[row, col].set_title(k)
    axes[row, col].set_xticklabels(['5G+5G', '5G+LTE', 'B1B3\n+B7B8', 'B1B8\n+LTE', 'DBL'])

# 顯示圖表
plt.tight_layout()
plt.show()